<a href="https://colab.research.google.com/github/entangelk/study_data_analytics/blob/main/docs/quests/CDAs/CDA_Categorical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 한글 폰트 설치 및 시각화

In [ ]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 런타임메뉴에서 리스타트 세션 해줘야함

- 한글 폰트 설치가 끝났으면 리스타트 잊지말라구!

In [1]:
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import pandas as pd

In [4]:
# 유니코드 마이너스 사용 설정
plt.rcParams['axes.unicode_minus'] = False

In [5]:
import scipy.stats as stats

In [6]:
get_data = pd.read_csv('fruitstore_saleshistory.csv')
get_data

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705566,2018-06-30,토,6,26,146297344,정상회원,수산,건어물,멸치류,국물멸치(300g),1.0,수지구,성복동,여,51.0,50대,9189,매장A,NaN,17:56
705567,2018-06-30,토,6,26,1532985472,정상회원,수산,해조,미역/다시마,자른다시마(100g),1.0,수지구,동천동,여,63.0,60대,5169,매장A,NaN,14:37
705568,2018-06-30,토,6,26,1532985472,정상회원,반찬,두부/유부,두부,두부(1모:420g),2.0,수지구,동천동,여,63.0,60대,7466,매장A,NaN,14:37
705569,2018-06-30,토,6,26,1209779776,정상회원,간식,과자,스낵,하늘바다새우(60g),1.0,수지구,동천동,여,41.0,40대,3542,매장A,NaN,18:04


## 이상치, 결측치 확인


### 결측치 확인

In [7]:
get_data.isnull().sum()
# 반품 원거래 일자의 숫자가 800개정도이므로 반품_원거래일자에 결측치가 있는것은 정상(반품한 제품)

공급일자             0
요일               0
공급월              0
공급주차             0
회원번호             0
조합원상태            0
물품대분류            0
물품중분류            0
물품소분류            0
물품명              0
구매수량             0
주소-구             0
주소-동             0
성별               5
연령              33
연령대             33
구매금액             0
구매매장             0
반품_원거래일자    694337
구매시각             0
dtype: int64

In [8]:
get_data=get_data.dropna(subset=['성별','연령','연령대','구매금액','구매매장','구매시각'])
# 나머지 결측치 제거

### 이상치 확인

In [9]:
get_data['성별'].unique()
# 성별 정상

array(['여', '남'], dtype=object)

In [10]:
get_data['연령대'].unique()
# 연령대 정상

array(['40대', '30대이하', '50대', '60대', '70대이상'], dtype=object)

In [11]:
get_data['물품대분류'].unique()
# 물품분류 정상

array(['과실', '채소', '축산물', '반찬', '간식', '수산', '주곡', '양념/가루', '음료', '생활용품',
       '서류', '건강', '잡곡', '기타', '주류'], dtype=object)

## X(범주형) / Y(범주형) 케이스


### 과일 유형(X)과 고객 세그먼트(Y) 사이에 연관성이 있습니까?
분석 방법: 카이제곱 독립성 검정을 사용하여 서로 다른 고객 세그먼트에 걸쳐 과일 유형의 판매 분포가 독립적인지 평가합니다.

- 과일 유형 = 물품소분류 중 과일 데이터 사용
- 고객 세그먼트 = 성별, 나이대 사용


In [12]:
get_data_fr_cu = get_data[['물품대분류','물품소분류','성별','연령대']].query('물품대분류 == "과실"')
get_data_fr_cu

,물품대분류,물품소분류,성별,연령대
0,과실,사과,여,40대
30,과실,땅콩,남,70대이상
62,과실,건대추,여,50대
132,과실,귤(조생),여,60대
187,과실,귤(조생),여,40대
...,...,...,...,...
705342,과실,황매실,남,50대
705354,과실,황매실,여,40대
705429,과실,산딸기,여,50대
705467,과실,사과,여,30대이하


In [13]:
get_data_fr_cu_pivot_sex = pd.pivot_table(get_data_fr_cu[['물품소분류','성별']], index='물품소분류', columns='성별', aggfunc=len)
get_data_fr_cu_pivot_sex

성별,남,여
물품소분류,,
건대추,24,264
곶감,21,446
귤(비가림),17,210
귤(조생),82,1152
귤만감,96,1676
냉동건조과일,8,25
단감,56,707
땅콩,120,1433
레몬,6,145


In [14]:
stats.chi2_contingency(observed=get_data_fr_cu_pivot_sex, correction=True)

Chi2ContingencyResult(statistic=48.62100488402738, pvalue=0.0008977173119671627, dof=22, expected_freq=array([[1.85893639e+01, 2.69410636e+02],
       [3.01431700e+01, 4.36856830e+02],
       [1.46520334e+01, 2.12347967e+02],
       [7.96502607e+01, 1.15434974e+03],
       [1.14376225e+02, 1.65762377e+03],
       [2.13003128e+00, 3.08699687e+01],
       [4.92489051e+01, 7.13751095e+02],
       [1.00240563e+02, 1.45275944e+03],
       [9.74650678e+00, 1.41253493e+02],
       [5.86726799e+01, 8.50327320e+02],
       [8.17157456e+01, 1.18428425e+03],
       [2.58185610e-01, 3.74181439e+00],
       [3.01431700e+01, 4.36856830e+02],
       [4.21746194e+02, 6.11225381e+03],
       [3.00140772e+01, 4.34985923e+02],
       [7.42283629e+00, 1.07577164e+02],
       [5.80917623e-01, 8.41908238e+00],
       [4.38915537e+01, 6.36108446e+02],
       [9.33340980e+01, 1.35266590e+03],
       [6.06736184e+00, 8.79326382e+01],
       [3.36286757e+01, 4.87371324e+02],
       [3.03368092e+00, 4.39663191e+

In [15]:
import numpy as np

In [16]:
get_data_fr_cu_pivot_age = pd.pivot_table(get_data_fr_cu[['물품소분류','연령대']], index='물품소분류', columns='연령대', aggfunc=len)
get_data_fr_cu_pivot_age = get_data_fr_cu_pivot_age.replace(np.nan,0)
get_data_fr_cu_pivot_age

연령대,30대이하,40대,50대,60대,70대이상
물품소분류,,,,,
건대추,29.0,88.0,66.0,62.0,43.0
곶감,42.0,166.0,140.0,67.0,52.0
귤(비가림),25.0,86.0,57.0,25.0,34.0
귤(조생),168.0,544.0,308.0,120.0,94.0
귤만감,192.0,579.0,541.0,289.0,171.0
냉동건조과일,3.0,12.0,8.0,2.0,8.0
단감,74.0,283.0,206.0,112.0,88.0
땅콩,133.0,510.0,471.0,290.0,149.0
레몬,18.0,53.0,37.0,27.0,16.0


In [18]:
stats.chi2_contingency(observed=get_data_fr_cu_pivot_age, correction=True)

Chi2ContingencyResult(statistic=447.88209526525463, pvalue=1.2890673521507695e-49, dof=88, expected_freq=array([[3.42807091e+01, 9.60700730e+01, 7.65947862e+01, 4.74944734e+01,
        3.35599583e+01],
       [5.55871220e+01, 1.55780292e+02, 1.24200574e+02, 7.70136079e+01,
        5.44184046e+01],
       [2.70198644e+01, 7.57218978e+01, 6.03715850e+01, 3.74348801e+01,
        2.64517727e+01],
       [1.46883316e+02, 4.11633577e+02, 3.28187383e+02, 2.03500626e+02,
        1.43795099e+02],
       [2.10921585e+02, 5.91097810e+02, 4.71270699e+02, 2.92222941e+02,
        2.06486966e+02],
       [3.92799791e+00, 1.10080292e+01, 8.77648592e+00, 5.44207508e+00,
        3.84541189e+00],
       [9.08200730e+01, 2.54518978e+02, 2.02922993e+02, 1.25827372e+02,
        8.89105839e+01],
       [1.84853962e+02, 5.18044526e+02, 4.13026747e+02, 2.56107351e+02,
        1.80967414e+02],
       [1.79735662e+01, 5.03700730e+01, 4.01590719e+01, 2.49016163e+01,
        1.75956726e+01],
       [1.08198488e+02

### 분석 결과 (기준 : pvalue=0.05)
- 통계적 결론 :
  + 과일 - 성별 : 대립가설이 성립(귀무 가설 기각)
  + 과일 - 나이대 : 귀무가설이 성립(대립 가설 기각)
- 사실 결론 :
  + 과일 - 성별 : 성별에 따라 과일 종류에 대한 구매와 관계가 있다.
  + 과일 - 나이대 : 나이대에 따라 과일 종류에 대한 구매와 관계가 없다.